# Project Two Code 

In [18]:
import numpy as np
import nltk
from nltk.corpus import stopwords
import pandas as pd
import matplotlib.pyplot as plt
from collections import Counter
from itertools import chain
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from fuzzywuzzy import fuzz
from sklearn.linear_model import LogisticRegression
from transformers import pipeline

## Web Scraping NYT Articles Using NYT API 

In [19]:
import requests as req
import time
API_KEY='9kiWZKsGFtTkcTD4yYkyxWAKehiTKkzR' # your API key

In [21]:
import calendar

# Create an empty list to store articles
articles = []

# Loop through every month from Jan 2020 to Feb 2024
for year in range(2020, 2025):
    for month in range(1, 13):
        # Calculate the number of days in the current month
        num_days = calendar.monthrange(year, month)[1]

        # Construct the begin_date and end_date parameters for the query
        begin_date = f"{year:04d}-{month:02d}-01"
        end_date = f"{year:04d}-{month:02d}-{num_days:02d}"

        # Loop through 5 pages (max 10 results per page, hence 50 articles)
        for i in range(5):
            try:
                # Construct the URL
                url = f'https://api.nytimes.com/svc/search/v2/articlesearch.json?q=artificial+intelligence&api-key={API_KEY}&sort=relevance&begin_date={begin_date}&end_date={end_date}&page={i+1}'

                # Make the request and get the response
                response = req.get(url).json()
                # Extract relevant information from the response and append to articles list
                docs = response['response']['docs']
                for doc in docs:
                    filteredDoc = {}
                    filteredDoc['title'] = doc['headline']['main']
                    filteredDoc['abstract'] = doc['abstract']
                    filteredDoc['paragraph'] = doc['lead_paragraph']
                    filteredDoc['date'] = f"{year}-{month:02d}"
                    articles.append(filteredDoc)
                
                # Sleep to avoid exceeding the API rate limit
                time.sleep(10)
            except KeyError as e:
                print(f"No response while processing page {i+1} for {begin_date} - {end_date}: {e}")


No response while processing page 5 for 2020-05-01 - 2020-05-31: 'response'
No response while processing page 1 for 2020-07-01 - 2020-07-31: 'response'
No response while processing page 2 for 2020-07-01 - 2020-07-31: 'response'
No response while processing page 3 for 2020-08-01 - 2020-08-31: 'response'
No response while processing page 4 for 2020-09-01 - 2020-09-30: 'response'
No response while processing page 5 for 2020-10-01 - 2020-10-31: 'response'
No response while processing page 1 for 2020-11-01 - 2020-11-30: 'response'
No response while processing page 2 for 2020-12-01 - 2020-12-31: 'response'
No response while processing page 3 for 2021-01-01 - 2021-01-31: 'response'
No response while processing page 4 for 2021-01-01 - 2021-01-31: 'response'
No response while processing page 5 for 2021-02-01 - 2021-02-28: 'response'
No response while processing page 1 for 2021-04-01 - 2021-04-30: 'response'
No response while processing page 2 for 2021-05-01 - 2021-05-31: 'response'
No response 

KeyboardInterrupt: 

In [22]:
# Create a DataFrame from the list of articles
nyt_ai = pd.DataFrame(data=articles)

In [23]:
print(nyt_ai['date'].unique())

['2020-01' '2020-02' '2020-03' '2020-04' '2020-05' '2020-06' '2020-08'
 '2020-09' '2020-10' '2020-11' '2020-12' '2021-01' '2021-02' '2021-03'
 '2021-04' '2021-05' '2021-06' '2021-07' '2021-08' '2021-09' '2021-10'
 '2021-11' '2021-12' '2022-01' '2022-02' '2022-03' '2022-04' '2022-05'
 '2022-06' '2022-07' '2022-08' '2022-09' '2022-10' '2022-11' '2022-12'
 '2023-01' '2023-02' '2023-03' '2023-04' '2023-05' '2023-06' '2023-07'
 '2023-08' '2023-09' '2023-10' '2023-11' '2023-12' '2024-01' '2024-02'
 '2024-03']


In [24]:
nyt_ai.shape

(1402, 4)

In [25]:
nyt_ai.to_csv('r1_scraped_nyt.csv')

## Text Cleaning

## Preliminary Sentiment Analysis 